# Genetic algorithm-based heuristic for feature selection in credit risk assessment

First we import libraries ,we will be needed through this notebook : 

In [273]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import adam
from keras.optimizers import SGD
from numpy.random import rand as random

## Configes

In [332]:
config = { 
    'maxFeatureNum': 12,
    'minFeatureNum': 5,
    'populationSize': 50,
    'initProb': .5,
    'crossoverProb': .9
}

In [240]:
def accuracy(predictions, labels):
#     print(predictions)
#     print(labels)
    return (100.0 * np.sum(predictions.reshape(-1) == labels.reshape(-1))
          / predictions.shape[0])

## Implementation of Neuralnetwork architecture

In [109]:
def tens_train(X,Y,hiddenNum=40,trainCycles=450,lr=.1,m=.5) :
    """Train one layer feedforward neural network
    Args :
       X : training data
       Y : training label
       hiddenNum : number of hidden units of hidden layer
       trainCycles : number of training cycles
       lr : learning rate of nueral network
       m: momentum of neural network
    Returns :
       'float' accuracy
    """
    
    hidden_size = hiddenNum
    batch_size = len(X)
    attr_size = X.shape[1]
    num_steps = 450
    num_labels = 1
    graph = tf.Graph()
    
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, attr_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
#         tf_valid_dataset = tf.constant(valid_dataset)
#         tf_test_dataset = tf.constant(test_dataset)

        # layer1
        weights_1 = tf.Variable(
        tf.truncated_normal([attr_size, hidden_size]))
        biases_1 = tf.Variable(tf.zeros([hidden_size]))
        layer_1 = tf.nn.sigmoid(tf.matmul(tf_train_dataset, weights_1) + biases_1)

        # layer2  
        weights_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        # Training computation.
        layer_2 = tf.matmul(layer_1,weights_2) + biases_2
  
        # calculating Loss
        loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_train_labels, logits=layer_2))

        # Optimizer.
        optimizer = tf.train.MomentumOptimizer(lr,m).minimize(loss)
        train_prediction = tf.nn.sigmoid(layer_2)
        
        
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")

        
        for step in range(trainCycles):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (X.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = X[offset:(offset + batch_size), :]
            batch_labels = Y[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 1 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            

In [195]:
def tens_train(X,Y,hiddenNum=40,trainCycles=450,lr=.1,m=.5) :
    """Train one layer feedforward neural network
    Args :
       X : training data
       Y : training label
       hiddenNum : number of hidden units of hidden layer
       trainCycles : number of training cycles
       lr : learning rate of nueral network
       m: momentum of neural network
    Returns :
       'float' accuracy
    """
    
    hidden_size = hiddenNum
    batch_size = len(X)
    attr_size = X.shape[1]
    num_steps = 450
    num_labels = Y.shape[1]
    graph = tf.Graph()
    
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, attr_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
#         tf_valid_dataset = tf.constant(valid_dataset)
#         tf_test_dataset = tf.constant(test_dataset)

        # layer1
        weights_1 = tf.Variable(
        tf.truncated_normal([attr_size, hidden_size]))
        biases_1 = tf.Variable(tf.zeros([hidden_size]))
        layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

        # layer2  
        weights_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        # Training computation.
        layer_2 = tf.matmul(layer_1,weights_2) + biases_2
  
        # calculating Loss
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=layer_2))

        # Optimizer.
        optimizer = tf.train.MomentumOptimizer(lr,m).minimize(loss)
        train_prediction = tf.nn.sigmoid(layer_2)
        
        
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        feed_dict = {tf_train_dataset : X, tf_train_labels : Y}
        _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        print("sd sMinibatch accuracy: %.1f%%" % accuracy(predictions, Y))
        for step in range(trainCycles):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
#             offset = (step * batch_size) % (X.shape[0] - batch_size)
            # Generate a minibatch.
#             batch_data = X[offset:(offset + batch_size), :]
#             batch_labels = Y[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : X, tf_train_labels : Y}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 1 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, Y))
            

In [241]:
x_train = pd.read_excel('GermanCreditInput.xls',header=None)
# x_train[[0,1,2,3,4,6,9,10,11,12,13,14,17,19,20,21,22,24,27,29]]
y_train = pd.read_excel('GermanCreditOutputClass1columnknn.xls',header=None)
y_train = (np.arange(2) == np.array(y_train)[:]).astype(np.float32)
tens_train(np.array(x_train),np.array(y_train),90,1000,.001,.1)
x_train

Initialized
sd sMinibatch accuracy: 119.6%
Minibatch loss at step 0: 167679.531250
Minibatch accuracy: 60.0%
Minibatch loss at step 1: 0.693016
Minibatch accuracy: 0.0%
Minibatch loss at step 2: 0.692297
Minibatch accuracy: 0.0%
Minibatch loss at step 3: 0.692120
Minibatch accuracy: 0.0%
Minibatch loss at step 4: 0.691987
Minibatch accuracy: 0.0%
Minibatch loss at step 5: 0.691860
Minibatch accuracy: 0.0%
Minibatch loss at step 6: 0.691734
Minibatch accuracy: 0.0%
Minibatch loss at step 7: 0.691610
Minibatch accuracy: 0.0%
Minibatch loss at step 8: 0.691488
Minibatch accuracy: 0.0%
Minibatch loss at step 9: 0.691368
Minibatch accuracy: 0.0%
Minibatch loss at step 10: 0.691249
Minibatch accuracy: 0.0%
Minibatch loss at step 11: 0.691132
Minibatch accuracy: 0.0%
Minibatch loss at step 12: 0.691025
Minibatch accuracy: 0.0%
Minibatch loss at step 13: 0.690935
Minibatch accuracy: 0.0%
Minibatch loss at step 14: 0.690846
Minibatch accuracy: 0.0%
Minibatch loss at step 15: 0.690757
Minibatch 

Minibatch loss at step 209: 0.675265
Minibatch accuracy: 0.0%
Minibatch loss at step 210: 0.675188
Minibatch accuracy: 0.0%
Minibatch loss at step 211: 0.675110
Minibatch accuracy: 0.0%
Minibatch loss at step 212: 0.675033
Minibatch accuracy: 0.0%
Minibatch loss at step 213: 0.674955
Minibatch accuracy: 0.0%
Minibatch loss at step 214: 0.674879
Minibatch accuracy: 0.0%
Minibatch loss at step 215: 0.674802
Minibatch accuracy: 0.0%
Minibatch loss at step 216: 0.674726
Minibatch accuracy: 0.0%
Minibatch loss at step 217: 0.674649
Minibatch accuracy: 0.0%
Minibatch loss at step 218: 0.674574
Minibatch accuracy: 0.0%
Minibatch loss at step 219: 0.674498
Minibatch accuracy: 0.0%
Minibatch loss at step 220: 0.674423
Minibatch accuracy: 0.0%
Minibatch loss at step 221: 0.674347
Minibatch accuracy: 0.0%
Minibatch loss at step 222: 0.674272
Minibatch accuracy: 0.0%
Minibatch loss at step 223: 0.674197
Minibatch accuracy: 0.0%
Minibatch loss at step 224: 0.674122
Minibatch accuracy: 0.0%
Minibatc

Minibatch loss at step 418: 0.661208
Minibatch accuracy: 0.0%
Minibatch loss at step 419: 0.661152
Minibatch accuracy: 0.0%
Minibatch loss at step 420: 0.661095
Minibatch accuracy: 0.0%
Minibatch loss at step 421: 0.661039
Minibatch accuracy: 0.0%
Minibatch loss at step 422: 0.660982
Minibatch accuracy: 0.0%
Minibatch loss at step 423: 0.660926
Minibatch accuracy: 0.0%
Minibatch loss at step 424: 0.660870
Minibatch accuracy: 0.0%
Minibatch loss at step 425: 0.660814
Minibatch accuracy: 0.0%
Minibatch loss at step 426: 0.660758
Minibatch accuracy: 0.0%
Minibatch loss at step 427: 0.660702
Minibatch accuracy: 0.0%
Minibatch loss at step 428: 0.660646
Minibatch accuracy: 0.0%
Minibatch loss at step 429: 0.660591
Minibatch accuracy: 0.0%
Minibatch loss at step 430: 0.660535
Minibatch accuracy: 0.0%
Minibatch loss at step 431: 0.660479
Minibatch accuracy: 0.0%
Minibatch loss at step 432: 0.660423
Minibatch accuracy: 0.0%
Minibatch loss at step 433: 0.660368
Minibatch accuracy: 0.0%
Minibatc

Minibatch loss at step 667: 0.647176
Minibatch accuracy: 0.0%
Minibatch loss at step 668: 0.647126
Minibatch accuracy: 0.0%
Minibatch loss at step 669: 0.647076
Minibatch accuracy: 0.0%
Minibatch loss at step 670: 0.647027
Minibatch accuracy: 0.0%
Minibatch loss at step 671: 0.646978
Minibatch accuracy: 0.0%
Minibatch loss at step 672: 0.646930
Minibatch accuracy: 0.0%
Minibatch loss at step 673: 0.646882
Minibatch accuracy: 0.0%
Minibatch loss at step 674: 0.646834
Minibatch accuracy: 0.0%
Minibatch loss at step 675: 0.646786
Minibatch accuracy: 0.0%
Minibatch loss at step 676: 0.646738
Minibatch accuracy: 0.0%
Minibatch loss at step 677: 0.646691
Minibatch accuracy: 0.0%
Minibatch loss at step 678: 0.646645
Minibatch accuracy: 0.0%
Minibatch loss at step 679: 0.646597
Minibatch accuracy: 0.0%
Minibatch loss at step 680: 0.646551
Minibatch accuracy: 0.0%
Minibatch loss at step 681: 0.646495
Minibatch accuracy: 0.0%
Minibatch loss at step 682: 0.646411
Minibatch accuracy: 0.0%
Minibatc

Minibatch loss at step 903: 0.637193
Minibatch accuracy: 0.0%
Minibatch loss at step 904: 0.637162
Minibatch accuracy: 0.0%
Minibatch loss at step 905: 0.637130
Minibatch accuracy: 0.0%
Minibatch loss at step 906: 0.637098
Minibatch accuracy: 0.0%
Minibatch loss at step 907: 0.637067
Minibatch accuracy: 0.0%
Minibatch loss at step 908: 0.637035
Minibatch accuracy: 0.0%
Minibatch loss at step 909: 0.637004
Minibatch accuracy: 0.0%
Minibatch loss at step 910: 0.636972
Minibatch accuracy: 0.0%
Minibatch loss at step 911: 0.636941
Minibatch accuracy: 0.0%
Minibatch loss at step 912: 0.636909
Minibatch accuracy: 0.0%
Minibatch loss at step 913: 0.636878
Minibatch accuracy: 0.0%
Minibatch loss at step 914: 0.636847
Minibatch accuracy: 0.0%
Minibatch loss at step 915: 0.636816
Minibatch accuracy: 0.0%
Minibatch loss at step 916: 0.636784
Minibatch accuracy: 0.0%
Minibatch loss at step 917: 0.636753
Minibatch accuracy: 0.0%
Minibatch loss at step 918: 0.636722
Minibatch accuracy: 0.0%
Minibatc

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0,6,4,0,0,0,1,0,0,1169,...,0,67,0,0,1,2,2,1,1,0
1,1,48,2,0,0,0,1,0,0,5951,...,0,22,0,0,1,1,2,1,0,0
2,3,12,4,0,0,0,0,1,0,2096,...,0,49,0,0,1,1,1,2,0,0
3,0,42,2,0,0,1,0,0,0,7882,...,0,45,0,0,0,1,2,2,0,0
4,0,24,3,1,0,0,0,0,0,4870,...,1,53,0,0,0,2,2,2,0,0
5,3,36,2,0,0,0,0,1,0,9055,...,1,35,0,0,0,1,1,2,1,0
6,3,24,2,0,0,1,0,0,0,2835,...,0,53,0,0,1,1,2,1,0,0
7,1,36,2,0,1,0,0,0,0,6948,...,0,35,0,1,0,1,3,1,1,0
8,3,12,2,0,0,0,1,0,0,3059,...,0,61,0,0,1,1,1,1,0,0
9,1,30,4,1,0,0,0,0,0,5234,...,0,28,0,0,1,2,3,1,0,0


In [164]:
np.array([[2],[4],[6]])[2,None]

array([[6]])

In [262]:
def simple_train(X,Y,hiddenNum=40,trainCycles=450,lr=.1,m=.5) :
    """Train one layer feedforward neural network
    Args :
       X : training data
       Y : training label
       hiddenNum : number of hidden units of hidden layer
       trainCycles : number of training cycles
       lr : learning rate of nueral network
       m: momentum of neural network
    Returns :
       'float' accuracy
    """
    model = Sequential()
    model.add(Dense(hiddenNum, input_dim=X.shape[1], kernel_initializer='normal', activation='selu'))
#     model.add(Dense(hiddenNum, kernel_initializer='normal', activation='selu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    sgd = SGD(lr=lr, momentum=m)
    ADAM = adam(lr=lr)
    # loss could be "mse" too
    model.compile(loss='binary_crossentropy',metrics=['binary_accuracy'],optimizer=sgd)
    for i in range(trainCycles) :
        combined = list(zip(X, Y))
        random.shuffle(combined)
        X[:], Y[:] = zip(*combined)
        model.fit(X,Y,batch_size=len(X))
    return model

In [263]:
x_train = pd.read_excel('GermanCreditInput.xls',header=None)
y_train = pd.read_excel('GermanCreditOutputClass1columnknn.xls',header=None)
# y_train = (np.arange(2) == np.array(y_train)[:]).astype(np.float32)
X = np.append(x_train[y_train[0] == 1][:300], x_train[y_train[0] == 0][:300],axis=0)
Y = np.append(y_train[y_train[0] == 1][:300], y_train[y_train[0] == 0][:300],axis=0)
combined = list(zip(X, Y))
random.shuffle(combined)
X[:], Y[:] = zip(*combined)
# np.sum((np.array(y_train).reshape(-1) == 1).argmax())
# (np.array(y_train).reshape(-1) == 1)
simple_train(np.array(X),np.array(Y),30,450,.3,.9)
# x_train[y_train == 1]

Epoch 1/1
600/600 [==============================] - 1s 1ms/step - loss: 9.3716 - binary_accuracy: 0.2467
Epoch 1/1
600/600 [==============================] - 0s 10us/step - loss: 4.1370 - binary_accuracy: 0.7433
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 4.0295 - binary_accuracy: 0.7500
Epoch 1/1
600/600 [==============================] - 0s 11us/step - loss: 4.1370 - binary_accuracy: 0.7433
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 4.1638 - binary_accuracy: 0.7417
Epoch 1/1
600/600 [==============================] - 0s 10us/step - loss: 4.0295 - binary_accuracy: 0.7500
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 4.1370 - binary_accuracy: 0.7433
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 3.8952 - binary_accuracy: 0.7583
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 3.1968 - binary_accuracy: 0.8017
Epoch 1/1
600/600 [========================

Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 1.1283 - binary_accuracy: 0.9300
Epoch 1/1
600/600 [==============================] - 0s 7us/step - loss: 1.0208 - binary_accuracy: 0.9367
Epoch 1/1
600/600 [==============================] - 0s 11us/step - loss: 1.2357 - binary_accuracy: 0.9233
Epoch 1/1
600/600 [==============================] - 0s 11us/step - loss: 1.3432 - binary_accuracy: 0.9167
Epoch 1/1
600/600 [==============================] - 0s 7us/step - loss: 1.4506 - binary_accuracy: 0.9100
Epoch 1/1
600/600 [==============================] - 0s 6us/step - loss: 1.2894 - binary_accuracy: 0.9200
Epoch 1/1
600/600 [==============================] - 0s 11us/step - loss: 1.3700 - binary_accuracy: 0.9150
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 1.5312 - binary_accuracy: 0.9050
Epoch 1/1
600/600 [==============================] - 0s 7us/step - loss: 1.4506 - binary_accuracy: 0.9100
Epoch 1/1
600/600 [========================

Epoch 1/1
600/600 [==============================] - 0s 13us/step - loss: 2.0954 - binary_accuracy: 0.8700
Epoch 1/1
600/600 [==============================] - 0s 9us/step - loss: 2.2028 - binary_accuracy: 0.8633
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 2.0685 - binary_accuracy: 0.8717
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 2.3909 - binary_accuracy: 0.8517
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 1.7461 - binary_accuracy: 0.8917
Epoch 1/1
600/600 [==============================] - 0s 9us/step - loss: 1.7193 - binary_accuracy: 0.8933
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 1.6924 - binary_accuracy: 0.8950
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 1.6118 - binary_accuracy: 0.9000
Epoch 1/1
600/600 [==============================] - 0s 7us/step - loss: 1.9342 - binary_accuracy: 0.8800
Epoch 1/1
600/600 [==========================

Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 3.9489 - binary_accuracy: 0.7550
Epoch 1/1
600/600 [==============================] - 0s 6us/step - loss: 3.6534 - binary_accuracy: 0.7733
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 3.8146 - binary_accuracy: 0.7633
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 3.9221 - binary_accuracy: 0.7567
Epoch 1/1
600/600 [==============================] - 0s 10us/step - loss: 4.2176 - binary_accuracy: 0.7383
Epoch 1/1
600/600 [==============================] - 0s 7us/step - loss: 4.3787 - binary_accuracy: 0.7283
Epoch 1/1
600/600 [==============================] - 0s 6us/step - loss: 4.6742 - binary_accuracy: 0.7100
Epoch 1/1
600/600 [==============================] - 0s 10us/step - loss: 4.4862 - binary_accuracy: 0.7217
Epoch 1/1
600/600 [==============================] - 0s 9us/step - loss: 4.2176 - binary_accuracy: 0.7383
Epoch 1/1
600/600 [=========================

Epoch 1/1
600/600 [==============================] - 0s 7us/step - loss: 2.7132 - binary_accuracy: 0.8317
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 2.3640 - binary_accuracy: 0.8533
Epoch 1/1
600/600 [==============================] - 0s 9us/step - loss: 2.2028 - binary_accuracy: 0.8633
Epoch 1/1
600/600 [==============================] - 0s 14us/step - loss: 2.3371 - binary_accuracy: 0.8550
Epoch 1/1
600/600 [==============================] - 0s 7us/step - loss: 2.0416 - binary_accuracy: 0.8733
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 2.0685 - binary_accuracy: 0.8717
Epoch 1/1
600/600 [==============================] - 0s 6us/step - loss: 2.0416 - binary_accuracy: 0.8733
Epoch 1/1
600/600 [==============================] - 0s 7us/step - loss: 2.0148 - binary_accuracy: 0.8750
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 2.0685 - binary_accuracy: 0.8717
Epoch 1/1
600/600 [==========================

Epoch 1/1
600/600 [==============================] - 0s 9us/step - loss: 5.2921 - binary_accuracy: 0.6717
Epoch 1/1
600/600 [==============================] - 0s 7us/step - loss: 5.9100 - binary_accuracy: 0.6333
Epoch 1/1
600/600 [==============================] - 0s 11us/step - loss: 6.0711 - binary_accuracy: 0.6233
Epoch 1/1
600/600 [==============================] - 0s 10us/step - loss: 6.4204 - binary_accuracy: 0.6017
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 6.9576 - binary_accuracy: 0.5683
Epoch 1/1
600/600 [==============================] - 0s 11us/step - loss: 6.3666 - binary_accuracy: 0.6050
Epoch 1/1
600/600 [==============================] - 0s 11us/step - loss: 6.5010 - binary_accuracy: 0.5967
Epoch 1/1
600/600 [==============================] - 0s 8us/step - loss: 6.4741 - binary_accuracy: 0.5983
Epoch 1/1
600/600 [==============================] - 0s 12us/step - loss: 6.4204 - binary_accuracy: 0.6017
Epoch 1/1
600/600 [======================

In [330]:
class Chromosome : 
    def __init__(self,genNum,trueProb) :
        self.chrom = np.array([True if random() > trueProb else False for _ in range(genNum)])
        self.fitness = None

    def __iter__(self) :
        return np.nditer(self.chrom)
        
    def mutate(self,prob) :
        self.chrom = np.vectorize(lambda x:x if random() > prob else not x)(self.chrom)
        
    def calculateFitness(X,Y,hiddenNum=40,trainCycles=450,lr=.1,m=.5) :
        """Train one layer feedforward neural network
        Args :
           X : training data
           Y : training label
           hiddenNum : number of hidden units of hidden layer
           trainCycles : number of training cycles
           lr : learning rate of nueral network
           m: momentum of neural network
        Returns :
           'float' accuracy
        """
        model = Sequential()
        model.add(Dense(hiddenNum, input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
        ADAM = adam(lr=lr)
        # loss could be "mse" too
        model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=ADAM)
        model.fit(X,Y,epochs=trainCycles)
        return model.accuricy

In [335]:
class Population :
    def __init__(self,popSize,genNum,trueProb,X,Y) :
        self.pop = np.array([])
        self.popSize = popSize
        self.genNum = genNum
        self.trueProb = trueProb
        self.X = X
        self.Y = Y
        for _ in range(popSize) : # 
            np.append(self.pop,Chromosome(genNum,trueProb))
        
    def __iter__(self) :
        return np.nditer(self.pop)
    
    def mutate(self,prob) :
        for chrom in self :
            chrom.mutate(prob)
    
    def crossover(self,prob) :
        newPop = Population(self.popSize,self.genNum,self.trueProb,self.X,self.Y)
        for i in range(len(self.pop)) :
            rand_prob = np.random.random()
            if rand_prob > prob :
                first = self.pop[np.random.randint(len(self.pop))].chrom
                second = self.pop[np.random.randint(len(self.pop))].chrom
                point = random.randint(len(first))
                newPop.pop[i].chrom = np.append(first[:point] + second[point:])
            else :
                newPop.pop[i].chrom = np.append(first[:point] + second[point:])
        return newPop

In [336]:
p = Population(config['populationSize'],12,config['initProb'],[10,10],[2,20])
config = { 
    'maxFeatureNum': 12,
    'minFeatureNum': 5,
    'populationSize': 50,
    'initProb': .5,
    'crossoverProb': .9
}

In [20]:
class GeneticAlgorithm :
    def __init__(self,initPopSize,genNum,trueProb) :
        self.bestChrom = None
        
    def run(self,mProb,cProb) :
        